<a href="https://colab.research.google.com/github/oceanumeric/ESL/blob/main/ch2/Ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2 Overview of Supervised Learning

Like the title indicates, this chapter gives the overview of supervised learning:


* Variable Types and Terminology
* Two Simple Approaches to Prediction
    * Least Squares
    * Nearest Neighbors 
* Statistical Decision Theory
* Local Methods in High Dimensions
* Statistical Models, Supervised Learning and Function Approximation
* Structured Regression Models
* Classes of Restricted Estimators
* Model Selection and the Bias–Variance Tradeoff

The goal of supervised learning is to predict $Y$ given $X$, where $X$ is called
input and $Y$ is called output. One should be aware of variable types:

* continuous variable
* discrete variable
    * categorical
    * ordered categorical 


## Two Simple Approaches to Prediction

For the linear model, the estimation 

## References

1. [Expected prediction error](https://stats.stackexchange.com/questions/92180/expected-prediction-error-derivation)